# Investigate optimal number of areas

format: ../results/opt_num_areas/pca_kmeans_{i}.csv

In [1]:
# FULDSTÆNDIG AUTOMATISERING 
mads_kører_kode = True

In [2]:
from plotter import Plotter

In [3]:
if mads_kører_kode:
    plotter = Plotter('C:/Users/madss/Aarhus Universitet/CompTek/6. Semester/Bachelor/github/FEPCEU/notebooks/results/opt_num_areas')
else:
    # Lav din plotter som passer med mappen til results
    #plotter = Plotter('some_path/to/folder')
    pass

In [4]:
data = plotter.load_csv_files('pca_kmeans')
len(data) # Should equal number of files in folder otherwise something is wrong

27

In [5]:
plotter.plot_metrics('MAE', f'Optimal number of areas included', y_lim=[0,1000])

## Fixing first ARIMA value

In [6]:
import pandas as pd
import numpy as np
import random

def compute_model_mean(data_frames, model_name='ARIMA', metric='MAE', param='mean'):
    """
    Compute the mean of {metric}_mean values for a specific model across multiple DataFrames.

    Args:
        data_frames (list of pd.DataFrame): List of DataFrames with model data.
        model_name (str): Model to filter (e.g., 'ARIMA').
        metric (str): Metric base name (default is 'MAE').

    Returns:
        float: The average {metric}_mean across all matching entries.
    """
    values = []

    for i, df in enumerate(data_frames):
        if i < 2:
            continue
        filtered = df[df['model'] == model_name]
        values.extend(filtered[f'{metric}_{param}'].values)

    return sum(values) / len(values) if values else None


def compute_mean_and_variance_of_variance(data_frames, model_name='ARIMA', metric='MAE', param='mean'):
    """
    Compute the mean of {metric}_mean and variance of variances ({metric}_std^2)
    for a specific model across multiple DataFrames (skipping first two).

    Returns:
        tuple: (mean_of_means, variance_of_variances)
    """
    means = []
    variances = []

    for i, df in enumerate(data_frames):
        if i < 2:
            continue
        filtered = df[df['model'] == model_name]

        # collect means
        means.extend(filtered[f'{metric}_mean'].values)
        # collect variances from std squared
        variances.extend(filtered[f'{metric}_{'std'}'].values)

    if not means or not variances:
        return None, None

    mean_of_means = np.mean(means)
    variance_of_variances = np.var(variances, ddof=1)  # sample variance

    return mean_of_means, variance_of_variances



arima_MAE_mean = compute_model_mean(data, 'ARIMA', metric='RMSE', param='mean')
arima_MAE_std = compute_model_mean(data, 'ARIMA', metric='RMSE', param='std')

me, st = compute_mean_and_variance_of_variance(data, 'ARIMA', metric='MAE')

arima_new_mean1 = random.normalvariate(arima_MAE_mean, arima_MAE_std)
arima_new_std = random.normalvariate(arima_MAE_std, st)

arima_new_mean1, arima_new_std




(np.float64(176.8974220440179), np.float64(18.142859090673383))